<h1>1.Descarga de información en Ensembl Plants</h1>

In [1]:
%load_ext rpy2.ipython

In [3]:
%%time
%%R

library(RMySQL)
library(curl)
library(biomaRt)
library(stringr)

cat("Iniciando proceso...\n")

dsn_database = "tesis2"
dsn_hostname = "localhost"
dsn_port = 3306
dsn_uid = "tesis2"
dsn_pwd = "tesis2"

conn = dbConnect(MySQL(), user=dsn_uid, password=dsn_pwd, dbname=dsn_database, host=dsn_hostname)
rs = dbSendQuery(conn, "SELECT id_especie, especie FROM maestra_especies m WHERE especie IN (SELECT c1.especie FROM conteo_lncrna_especie c1 LEFT JOIN conteo_lncrna_especie c2 ON c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'CantataDB' LEFT JOIN conteo_lncrna_especie c3 ON c1.especie = REPLACE(c3.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c3.fuente = 'GreeNC' WHERE c1.fuente = 'Ensembl' AND (c2.fuente IS NOT NULL OR c3.fuente IS NOT NULL)) AND NOT EXISTS (SELECT 1 FROM secuencias s WHERE s.id_especie = m.id_especie AND s.flg_pct = 1)")
especies = dbFetch(rs)
dbClearResult(rs)

ensembl_mart=useMart("plants_mart", host="plants.ensembl.org")
lista_datasets = listDatasets(ensembl_mart)

if (nrow(especies) > 0) {
    for(i in 1:nrow(especies)) {
        tryCatch(
            expr = {
                cat("\rProcesando especie #", i, " ", especies[i,2], "                                      ", sep = "")
                la_especie = especies[i,2]
                especie = paste("^", la_especie, sep="")
                dataset = lista_datasets[which(str_detect(lista_datasets$description, especie)), 1]
                ensembl_ds = useDataset(dataset,mart=ensembl_mart)
                genes = getBM(attributes=c('ensembl_transcript_id', 'transcript_exon_intron'), filters = 'transcript_biotype', values = 'protein_coding', mart = ensembl_ds)

                for(j in 1:nrow(genes)) {
                    if (j %% 1000 == 0) {
                        cat("\rProcesando especie #", i, " ", especies[i,2], " - procesadas ", j, " filas                     ", sep = "")
                    }
                    if (!(genes[j,2] == "Sequence unavailable" | str_length(genes[j,2]) > 65000)) {
                        query = sprintf("INSERT INTO secuencias (id_especie, cod_secuencia, secuencia, flg_pct, flg_seleccionado) VALUES (%d, '%s', '%s', 1, NULL)", especies[i,1], genes[j,1], genes[j,2])
                        dbSendQuery(conn, query)
                    }
                }
                cat("\rProcesando especie #", i, " ", especies[i,2], " - procesadas ", nrow(genes), " filas                     ", sep = "")
            },
            error = function(e) {
                cat("\rError al procesar la especie ", especies[i,2], "                                           \n", sep = "")
            }
        )
    }
}

dbDisconnect(conn)

cat("\rProceso finalizado...                                                                                       ")

Iniciando proceso...
Error al procesar la especie Amborella trichopoda                                           
Error al procesar la especie Arabidopsis lyrata                                           
Error al procesar la especie Brassica napus                                           
Error al procesar la especie Chlamydomonas reinhardtii                                           
Error al procesar la especie Glycine max                                           
Error al procesar la especie Gossypium raimondii                                           
Error al procesar la especie Hordeum vulgare                                           
Error al procesar la especie Medicago truncatula                                           
Error al procesar la especie Musa acuminata                                           
Error al procesar la especie Oryza brachyantha                                           
Error al procesar la especie Oryza nivara                                    

CPU times: user 8min 42s, sys: 6min 3s, total: 14min 46s
Wall time: 2h 26min 7s


<h2>1.1.Procesamiento de archivos descargados manualmente porque fallaron en la descarga automática</h2>

In [7]:
%%time
import mysql.connector
import os

#método para leer los transcriptomas de un archivo fasta de ensembl
def leer_transcriptomas_ensembl(archivo, cur, id_especie):
    cabecera = ""
    transcriptoma = ""
    f = open(archivo, "r")
    for linea in f:
        if linea.startswith(">"):
            if transcriptoma != "":
                if transcriptoma != "Sequence unavailable" and len(transcriptoma) <= 65000:
                    query = "INSERT INTO secuencias (id_especie, cod_secuencia, secuencia, flg_pct, flg_seleccionado) VALUES (%s, %s, %s, 1, NULL)"
                    cur.execute(query, (id_especie, cabecera, transcriptoma))
                transcriptoma = ""
            cabecera = linea.rstrip("\n").lstrip(">")
        else:
            transcriptoma += linea.rstrip("\n")
    if transcriptoma != "" and len(transcriptoma) <= 65000:
        if transcriptoma != "Sequence unavailable":
            query = "INSERT INTO secuencias (id_especie, cod_secuencia, secuencia, flg_pct, flg_seleccionado) VALUES (%s, %s, %s, 1, NULL)"
            cur.execute(query, (id_especie, cabecera, transcriptoma))
        transcriptoma = ""

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="tesis2",
   passwd="tesis2"
)

print("Iniciando proceso...") 

cur = conn.cursor()

directory = '../Semana 05/descarga_manual_ensembl'

for filename in os.listdir(directory):
    if filename.endswith(".fasta"):
        especie = os.path.splitext(filename)[0]
        query = "SELECT COUNT(*) FROM maestra_especies m WHERE especie IN (SELECT c1.especie FROM conteo_lncrna_especie c1 LEFT JOIN conteo_lncrna_especie c2 ON c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'CantataDB' LEFT JOIN conteo_lncrna_especie c3 ON c1.especie = REPLACE(c3.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c3.fuente = 'GreeNC' WHERE c1.fuente = 'Ensembl' AND (c2.fuente IS NOT NULL OR c3.fuente IS NOT NULL)) AND m.especie = REPLACE(%s, '_', ' ')"
        cur.execute(query, [especie])
        result=cur.fetchone()
        if result[0] > 0:
            print("\rProcesando especie ", especie, "                                        ", end=" ")
            query = "SELECT e.id_especie FROM maestra_especies e WHERE e.especie = REPLACE(%s, '_', ' ')"
            cur.execute(query, [especie])
            id_especie=cur.fetchone()
            query = "delete from secuencias where id_especie = %s"
            cur.execute(query, [id_especie[0]])
            leer_transcriptomas_ensembl(directory + "/" + filename, cur, id_especie[0])
            conn.commit()

conn.close()

print("\rProceso finalizado...                                                                                       ")

Iniciando proceso...
Proceso finalizado...                                                                                       
CPU times: user 5min 3s, sys: 36.9 s, total: 5min 40s
Wall time: 15min 42s


<h1>2.Descarga de información en CantataDB</h1>

In [8]:
%%time
import mysql.connector
import os

#método para leer los transcriptomas de un archivo fasta en cantatadb
def leer_transcriptomas_cantatadb(archivo, cur, id_especie):
    cabecera = ""
    transcriptoma = ""
    f = open(archivo, "r")
    for linea in f:
        if linea.startswith(">"):
            if transcriptoma != "":
                if len(transcriptoma) <= 65000:
                    query = "INSERT INTO secuencias (id_especie, cod_secuencia, secuencia, flg_pct, flg_seleccionado) VALUES (%s, %s, %s, 0, NULL)"
                    cur.execute(query, (id_especie, cabecera, transcriptoma))
                transcriptoma = ""
            cabecera = linea.rstrip("\n").lstrip(">")
        else:
            transcriptoma += linea.rstrip("\n")
    if transcriptoma != "":
        if len(transcriptoma) <= 65000:
            query = "INSERT INTO secuencias (id_especie, cod_secuencia, secuencia, flg_pct, flg_seleccionado) VALUES (%s, %s, %s, 0, NULL)"
            cur.execute(query, (id_especie, cabecera, transcriptoma))
        transcriptoma = ""

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="tesis2",
   passwd="tesis2"
)

print("Iniciando proceso...")

cur = conn.cursor()

directory = '../Semana 02/httpcantata.amu.edu.pldownload.php/Otros'

for filename in os.listdir(directory):
    if filename.endswith(".fasta"):
        especie = os.path.splitext(filename)[0].split("_lncrnas")[:-1][0]
        query = "SELECT COUNT(*) FROM maestra_especies m WHERE especie IN (SELECT c1.especie FROM conteo_lncrna_especie c1 LEFT JOIN conteo_lncrna_especie c2 ON c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'CantataDB' LEFT JOIN conteo_lncrna_especie c3 ON c1.especie = REPLACE(c3.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c3.fuente = 'GreeNC' WHERE c1.fuente = 'Ensembl' AND (c2.fuente IS NOT NULL OR c3.fuente IS NOT NULL)) AND NOT EXISTS (SELECT 1 FROM secuencias s WHERE s.id_especie = m.id_especie AND s.flg_pct = 0) AND m.especie = REPLACE(%s, '_', ' ') AND EXISTS (SELECT 1 FROM especies_seleccionadas es WHERE es.especie = m.especie AND fuente = %s)"
        cur.execute(query, (especie, "CantataDB"))
        result=cur.fetchone()
        if result[0] > 0:
            print("\rProcesando especie ", especie, "                                        ", end=" ")
            query = "SELECT e.id_especie FROM maestra_especies e WHERE e.especie = REPLACE(%s, '_', ' ')"
            cur.execute(query, [especie])
            id_especie=cur.fetchone()
            leer_transcriptomas_cantatadb(directory + "/" + filename, cur, id_especie[0])
            conn.commit()

conn.close()

print("\rProceso finalizado...                                                                                       ")

Iniciando proceso...
Proceso finalizado...                                                                                       
CPU times: user 10.3 s, sys: 2.49 s, total: 12.8 s
Wall time: 11min 36s


<h1>3.Descarga de información de GreenNC</h1>

In [10]:
%%time
import mysql.connector
import os

#método para leer los transcriptomas de un archivo fasta en cantatadb
def leer_transcriptomas_cantatadb(archivo, cur, id_especie):
    cabecera = ""
    transcriptoma = ""
    f = open(archivo, "r")
    for linea in f:
        if linea.startswith(">"):
            if transcriptoma != "":
                if len(transcriptoma) <= 65000:
                    query = "INSERT INTO secuencias (id_especie, cod_secuencia, secuencia, flg_pct, flg_seleccionado) VALUES (%s, %s, %s, 0, NULL)"
                    cur.execute(query, (id_especie, cabecera, transcriptoma))
                transcriptoma = ""
            cabecera = linea.rstrip("\n").lstrip(">")
        else:
            transcriptoma += linea.rstrip("\n")
    if transcriptoma != "":
        if len(transcriptoma) <= 65000:
            query = "INSERT INTO secuencias (id_especie, cod_secuencia, secuencia, flg_pct, flg_seleccionado) VALUES (%s, %s, %s, 0, NULL)"
            cur.execute(query, (id_especie, cabecera, transcriptoma))
        transcriptoma = ""

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="tesis2",
   passwd="tesis2"
)

print("Iniciando proceso...")

cur = conn.cursor()

directory = '../Semana 03/httpgreenc.sciencedesigners.com'

for filename in os.listdir(directory):
    if filename.endswith(".fasta"):
        especie = os.path.splitext(filename)[0]
        query = "SELECT COUNT(*) FROM maestra_especies m WHERE especie IN (SELECT c1.especie FROM conteo_lncrna_especie c1 LEFT JOIN conteo_lncrna_especie c2 ON c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'CantataDB' LEFT JOIN conteo_lncrna_especie c3 ON c1.especie = REPLACE(c3.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c3.fuente = 'GreeNC' WHERE c1.fuente = 'Ensembl' AND (c2.fuente IS NOT NULL OR c3.fuente IS NOT NULL)) AND NOT EXISTS (SELECT 1 FROM secuencias s WHERE s.id_especie = m.id_especie AND s.flg_pct = 0) AND m.especie = REPLACE(%s, '_', ' ') AND EXISTS (SELECT 1 FROM especies_seleccionadas es WHERE es.especie = m.especie AND fuente = %s)"
        cur.execute(query, (especie, "GreeNC"))
        result=cur.fetchone()
        if result[0] > 0:
            print("\rProcesando especie ", especie, "                                        ", end=" ")
            query = "SELECT id_especie FROM maestra_especies WHERE especie = REPLACE(%s, '_', ' ')"
            cur.execute(query, [especie])
            id_especie=cur.fetchone()
            leer_transcriptomas_cantatadb(directory + "/" + filename, cur, id_especie[0])
            conn.commit()

conn.close()

print("\rProceso finalizado...                                                                                       ")

Iniciando proceso...
Proceso finalizado...                                                                                       
CPU times: user 7.21 s, sys: 1.95 s, total: 9.16 s
Wall time: 6min 45s


<h1>Conteo de los resultados</h1>

In [11]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="tesis2",
   passwd="tesis2"
)
df = pd.read_sql_query("SELECT * FROM (SELECT m.especie, SUM(CASE WHEN s.flg_pct = 1 THEN 1 ELSE 0 END) AS PCT, SUM(CASE WHEN s.flg_pct = 0 THEN 1 ELSE 0 END) AS lncRNA FROM secuencias s JOIN maestra_especies m ON s.id_especie = m.id_especie GROUP BY m.especie) a WHERE a.PCT > 0 AND a.lncRNA > 0 AND a.especie NOT IN ('Zea mays') ORDER BY 3 DESC", conn)
display(df)

conn.close()

,especie,PCT,lncRNA
0,Triticum aestivum,133332.0,38820.0
1,Brassica napus,101039.0,12010.0
2,Oryza rufipogon,47421.0,10261.0
3,Trifolium pratense,41269.0,10179.0
4,Physcomitrella patens,86669.0,9690.0
5,Medicago truncatula,57581.0,9676.0
6,Manihot esculenta,41393.0,9504.0
7,Oryza nivara,48356.0,8955.0
8,Brassica rapa,41025.0,8501.0
9,Hordeum vulgare,234701.0,7970.0
